In [ ]:
seed_query_list = [
    "What courses are being offered for the {semester} semester?",
    "Which professor is taking the course {course_name}?",
    "What are the courses available related to {topic}?",
    "Can you please suggest online courses to take?",
    "Can you suggest research or seminar-based courses?",
    "Can you suggest courses without exams?",
    "Can you suggest courses which are not too hectic and easy to get good grades?",
    "What are the class timings for the course {course_name}?",
    "Are there any prerequisite courses for {course_name}?",
    "Has the course {course_name} been offered in the past? If so, could you summarize the reviews?",
    "How's the course {course_name} based on the reviews?",
    "How difficult is the course {course_name}?",
    "What courses are being offered by Professor {professor_name}?",
    "How is Professor {professor_name} in terms of grading?"
]

semester = "Spring 2025"

topics = [
    "Machine Learning",
    "Deep Learning",
    "Data Science",
    "Computer Systems",
    "Programming",
    "Development"
]

course_name = [
    'Foundations of Artificial Intelligence',
    'Algorithms',
    'Programming Design Paradigm',
    'Pattern Recognition and Computer Vision',
    'Natural Language Processing',
    'Deep Learning',
    'Machine Learning',
    'Information Retrieval',
    'Computer Systems',
    'Database Management',
    'Recitation for CS5010',
    'Foundations of Software Engineering',
    'Reinforcement Learning',
    'Computer/Human Interaction',
    'Web Development',
    'Compilers',
    'Mobile Application Development',
    'Discrete Structures',
    'Data Mining Techniques',
    'Fundamentals of Cloud Computing',
    'Object-Oriented Design',
    'Computer Systems'
]